In [2]:
import pandas as pd
import numpy as np

In [3]:
X_train = pd.read_csv('X_train_Kickstarter.csv')
X_test = pd.read_csv('X_test_Kickstarter.csv')
y_train = pd.read_csv('y_train_Kickstarter.csv')
y_test = pd.read_csv('y_test_Kickstarter.csv')